# Segmentation Notebook

### <font color='red'> After clicking on a code cell, press "Shift+Enter" to run the code, or click on the "Run" button in the toolbar above.<br>

### Replace "..." signs with the appropriate path to your data.
</font>

In [ ]:
from tapenade.preprocessing import (
    global_image_equalization,
    local_image_equalization,
)
from tapenade.preprocessing._preprocessing import change_array_pixelsize
from tapenade.segmentation._segment import segment_stardist
from tapenade.preprocessing.segmentation_postprocessing import remove_small_objects
import numpy as np
import tifffile
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.measure import regionprops

Enter the path to your data and to stardist model.

The mask is optional, in the case you use the local normalization method.

In [ ]:
path_stardist_model = ... #should finish by /tapenade-stardist

main_folder = ...
path_to_data = Path(main_folder) / ... #for example Path(main_folder) / "data/image.tif"
path_to_mask = Path(main_folder) / ... #for example Path(main_folder) / "data/mask.tif"
data = tifffile.imread(Path(path_to_data))
mask = tifffile.imread(Path(path_to_mask))

### Quick pre-processing
If your image already has the appropriate size and is normalized, you can skip this and directly go to the prediction.

Our StarDist model is trained on isotropic images of voxel size (0.62,0.62,0.62) µm/pix, which means it is trained to segment objects of the approximate size of 15pixels. Adjust your image size to match this pixel size and do not hesitate to try different sizes on a subset of your data to optimize the result.

The image given to StarDist should be normalized between 0 and 1. You can either use our local equalization method, that enhances contrast and show better performance in depth, or the classic global equalization.

In [ ]:
input_pixelsize = ... #ZYX, for example (1,0.2,0.2)
output_pixelsize = (0.62,0.62,0.62) #adjust if oversegmentation or undersegmentation
data_iso = change_array_pixelsize(data,input_pixelsize=input_pixelsize, output_pixelsize=output_pixelsize)
mask_iso = change_array_pixelsize(mask,input_pixelsize=input_pixelsize, output_pixelsize=output_pixelsize,order=0)

In [ ]:
data_normalized = local_image_equalization(image=data_iso,box_size= 25, perc_low=1,perc_high=99,mask=mask_iso)

In [ ]:
data_normalized = global_image_equalization(image=data_iso, perc_low=1,perc_high=99)

Check the resulting image on Napari if installed

In [ ]:
import napari
viewer=napari.Viewer()
viewer.add_image(data_iso,colormap='inferno')
viewer.add_image(data_normalized,colormap='inferno')
napari.run()

## Run the prediction using StarDist3D

In [ ]:
labels = segment_stardist(data_normalized, path_stardist_model)

Check the result on napari

In [ ]:
import napari
viewer=napari.Viewer()
viewer.add_image(data_normalized,colormap='inferno')
viewer.add_labels(labels)
napari.run()

Save if you are satisfied.
If not, you can adjust the pixel size or the normalization method.

In [ ]:
tifffile.imwrite(Path(main_folder)/'segmentation.tif',labels)

## Post-processing
#### 1 - Apply mask to remove out-of-sample detections

If some cells are detected outside of the image, you might want to delete what is outside your mask.

To create a mask of your image, use the preprocessing notebook or the napari plugin napati-tapenade-processing

In [ ]:
labels[mask==0]=0
tifffile.imwrite(Path(main_folder)/"labels_corrected.tif",labels)

#### 2 - Filter small volumes in the segmentation.

First, plot the histogram of cell volumes to evaluate the threshold

In [ ]:
props=regionprops(labels)
histo=[]
for prop in props :
    histo.append(np.sum(prop.area))

plt.hist(histo,bins=100)
plt.title('Histogram of cell volumes')

Then, remove the objects smaller than ```size_min```

In [ ]:
#choose the size to filter
size_min =1000
labels_filtered = remove_small_objects(labels,size_min)

print('Before filtering :',len(np.unique(labels)),'labels \nAfter filtering :',len(np.unique(labels_filtered)),'labels')

Visualize the result on napari

In [ ]:
import napari
viewer=napari.Viewer()
viewer.add_image(data_normalized)
viewer.add_labels(labels,name='labels_not_filtered')
viewer.add_labels(labels_filtered,name='labels_filtered')
napari.run()